Plot UMAP based on geneset of ~8000 disease genes with enrichment based on pleiotropy scores with hierarchy consideration of reactome pathways

## Target-pathway list

In [2]:
import pandas as pd

In [28]:
geneset = pd.read_csv("/Users/polina/genetics_gsea/data/geneset_annotated/geneset_reactome_2025_disease_v2.csv")

In [29]:
target_pathway_list = geneset[["ID", "approvedSymbol"]].copy()

target_pathway_list["ID"] = target_pathway_list["ID"].str.split(";")
target_pathway_exploded = target_pathway_list.explode("ID").dropna().reset_index(drop=True)

In [30]:
target_pathway_exploded

,ID,approvedSymbol
0,R-HSA-162582,CDKN2B
1,R-HSA-212436,CDKN2B
2,R-HSA-73857,CDKN2B
3,R-HSA-74160,CDKN2B
4,R-HSA-2173796,CDKN2B
...,...,...
13535,R-HSA-9937383,MRPL45
13536,R-HSA-9752946,OR10A6
13537,R-HSA-381753,OR10A6
13538,R-HSA-9752946,OR8D2


In [38]:
print(target_pathway_exploded.nunique())

ID                 407
approvedSymbol    4115
dtype: int64


## Pathway poincare embeddings

### Prepare pathway list with hierarchies

In [14]:
pathway_list = target_pathway_exploded["ID"].copy().drop_duplicates().reset_index(drop=True)

In [15]:
hieararchy = pd.read_csv("/Users/polina/genetics_gsea/data/gmt/Reactome_2025/Pathways_hierarchy_relationship.txt", sep="\t", header=None)
hieararchy.columns = ["parentId", "childId"]

In [18]:
pathway_list_hieararchy = pd.merge(pathway_list, hieararchy, left_on="ID", right_on="childId", how="left")[["ID", "parentId"]]

In [19]:
pathway_list_hieararchy

,ID,parentId
0,R-HSA-162582,NaN
1,R-HSA-212436,R-HSA-73857
2,R-HSA-73857,R-HSA-74160
3,R-HSA-74160,NaN
4,R-HSA-2173796,R-HSA-2173793
...,...,...
407,R-HSA-390666,R-HSA-375280
408,R-HSA-3229121,R-HSA-5663084
409,R-HSA-427359,R-HSA-5250941
410,R-HSA-1462054,R-HSA-1461973


### Prepare pathway embeddings

In [20]:
import math
import pandas as pd
from gensim.models.poincare import PoincareModel
from gensim.test.utils import temporary_file

def process_hierarchy_pathways(df, negative=10, epochs=100):
    """
    Learn embeddings for hierarchical pathways using Poincaré embeddings.
    
    Args:
        df (pd.DataFrame): Input DataFrame with columns:
                           - 'ID' (pathway/node)
                           - 'parentId' (parent pathway/node)
        negative (int): Maximum number of negative samples to use
        epochs (int): Number of training epochs
    
    Returns:
        pd.DataFrame: Embeddings with columns: ID, dim_0, dim_1, ...
    """
    # 1) Fill null parents with self (root nodes point to themselves)
    df = df.copy()
    df["parentId"] = df["parentId"].fillna(df["ID"])

    # 2) Count unique nodes
    nodes = pd.unique(df[["ID", "parentId"]].values.ravel())
    N = len(nodes)
    dims = max(2, math.ceil(math.log2(N)))
    print(f"Total distinct nodes N: {N}; Chosen d: {dims}")

    # Skip too small graphs
    if N < 3:
        print(f"Too few nodes ({N}) – returning empty DataFrame")
        return pd.DataFrame(columns=["ID"] + [f"dim_{i}" for i in range(dims)])

    # 3) Build edges (parent → child)
    edges = df[["parentId", "ID"]].drop_duplicates().values.tolist()

    # 4) Safe negative sampling
    def calculate_safe_negatives(requested, total_nodes):
        max_possible = total_nodes - 2  # conservative estimate
        safe_neg = min(requested, max(1, max_possible))
        print(f"Negative sampling: Requested {requested}, Safe maximum {max_possible}, Using {safe_neg}")
        return safe_neg

    neg = calculate_safe_negatives(negative, N)

    # 5) Train with retry logic
    model = None
    max_attempts = 2
    for attempt in range(max_attempts):
        try:
            model = PoincareModel(edges, negative=neg, size=dims)
            model.train(epochs=epochs)
            break
        except ValueError as e:
            if attempt == max_attempts - 1:
                print(f"Failed after {max_attempts} attempts: {str(e)}")
                return pd.DataFrame(columns=["ID"] + [f"dim_{i}" for i in range(dims)])
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            neg = max(1, neg - 2)
            print(f"Retrying with {neg} negatives")

    # 6) Extract embeddings
    emb = [(key, *model.kv[key]) for key in model.kv.index_to_key]
    pdf = pd.DataFrame(emb, columns=["ID"] + [f"dim_{i}" for i in range(dims)])
    return pdf


In [21]:
pem = process_hierarchy_pathways(pathway_list_hieararchy, negative=10, epochs=100)

Total distinct nodes N: 564; Chosen d: 10
Negative sampling: Requested 10, Safe maximum 562, Using 10


In [23]:
pem.to_csv("/Users/polina/genetics_gsea/data/pem/pem_reactome_2025_poincare.csv", index=False)

## Target embeddings

In [80]:
def create_target_embeddings(pathway_df: pd.DataFrame, target2pathway_df: pd.DataFrame) -> pd.DataFrame:
    """
    Generate target embeddings by averaging pathway embeddings per target.

    Parameters
    ----------
    pathway_df : pd.DataFrame
        Pathway embeddings with columns:
        - 'ID'
        - 'dim_*' (embedding dimensions)

    target2pathway_df : pd.DataFrame
        Exploded mapping between targets and pathways with columns:
        - 'approvedSymbol'
        - 'ID' (single pathway ID per row)

    Returns
    -------
    pd.DataFrame
        Averaged embeddings per target (without pathway IDs).
    """

    # Identify embedding columns
    embedding_columns = [c for c in pathway_df.columns if c.startswith("dim_")]

    # Join with pathway embeddings
    joined_df = target2pathway_df.merge(
        pathway_df,
        on="ID",
        how="inner"
    )

    # Average embeddings per target only
    averaged_df = (
        joined_df
        .groupby("approvedSymbol", as_index=False)
        .agg(
            **{col: ("{}".format(col), "mean") for col in embedding_columns}
        )
    )

    return averaged_df


In [81]:
tem = create_target_embeddings(pem, target_pathway_exploded)

In [82]:
tem.to_csv("/Users/polina/genetics_gsea/data/tem/tem_reactome_2025_poincare.csv", index=False)

## UMAP

### Prepare metadata and coordinate files

In [85]:
import os
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import umap.umap_ as umap
import hdbscan

def perform_umap_clustering_pandas(
    metadata_df,
    coords_df,
    output_dir,
    n_neighbors=10,
    min_dist=0.5,
    min_cluster_size=12,
    umap_dimensions=2
):
    """
    Performs UMAP dimensionality reduction and HDBSCAN clustering using Poincaré distance.
    Aligns metadata and coordinates by 'approvedSymbol', saves final result as TSV.
    """

    # Filter metadata
    metadata = metadata_df

    # Sanity checks
    if 'approvedSymbol' not in metadata.columns:
        raise ValueError("Metadata must contain 'approvedSymbol'")
    if coords_df.shape[1] <= 1:
        raise ValueError("Coordinates must have approvedSymbol + at least one dimension")

    # Rename first column to 'approvedSymbol' if not already
    coords_df = coords_df.rename(columns={coords_df.columns[0]: 'approvedSymbol'})

    # Convert coordinate columns to float
    coord_columns = coords_df.columns[1:]
    coords_df[coord_columns] = coords_df[coord_columns].astype(float)

    # Merge metadata and coordinates
    merged_df = pd.merge(metadata, coords_df, on='approvedSymbol', how='inner')
    print(f"Merged metadata and coordinates: {merged_df.shape[0]} entries.")

    # Extract embedding matrix
    embedding_matrix = merged_df[coord_columns].values

    # --- Poincaré distance ---
    def poincare_dist(u, v):
        norm_u = np.linalg.norm(u)
        norm_v = np.linalg.norm(v)
        norm_diff = np.linalg.norm(u - v)

        denom = (1 - norm_u ** 2) * (1 - norm_v ** 2)
        if denom <= 0:
            return np.inf

        argument = 1 + 2 * (norm_diff ** 2) / denom
        return np.arccosh(argument)

    def compute_poincare_distance_matrix(embedding_matrix):
        return squareform(pdist(embedding_matrix, metric=poincare_dist))

    # Check embeddings are inside Poincaré ball
    norms = np.linalg.norm(embedding_matrix, axis=1)
    if np.any(norms >= 1):
        raise ValueError("Some embeddings lie outside the Poincaré ball (norm >= 1).")

    distance_matrix = compute_poincare_distance_matrix(embedding_matrix)

    # --- UMAP ---
    reducer = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=umap_dimensions,
        metric='precomputed',
        random_state=42
    )
    embedding_umap = reducer.fit_transform(distance_matrix)

    # --- HDBSCAN ---
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=1,
        metric='precomputed'
    )
    cluster_labels = clusterer.fit_predict(distance_matrix)

    # Add UMAP + cluster results
    for dim in range(umap_dimensions):
        merged_df[f'UMAP {dim+1}'] = embedding_umap[:, dim]
    merged_df['cluster'] = cluster_labels

    # Drop original coordinates before saving
    output_df = merged_df.drop(columns=coord_columns)

    # Save
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, 'metadata_clusters_poincare.tsv')
    output_df.to_csv(output_file, sep='\t', index=False)

    print(f"✅ Updated metadata with clusters saved to: {output_file}")
    return output_file


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Prepare metadata file

In [86]:
nedometadata = tem[["approvedSymbol"]].copy()

In [87]:
metadata = pd.merge(nedometadata, geneset, on="approvedSymbol", how="left")[["approvedSymbol", "geneId", "ID", "Term", "uniqueDiseases"]]

In [102]:
tem

,approvedSymbol,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9
0,AARS2,-0.075705,0.199877,0.313767,0.014781,0.383315,-0.185769,0.139945,0.297208,-0.110708,-0.560982
1,ABCA1,0.504069,-0.252542,0.271395,-0.109036,-0.041243,0.177174,-0.341311,0.025945,0.314689,0.126597
2,ABCA4,0.444513,-0.239603,0.220231,-0.227197,-0.089924,0.330269,-0.497336,-0.178940,0.066368,0.367337
3,ABCA6,-0.207871,-0.206242,0.171302,-0.126589,-0.447826,-0.137853,0.252757,0.354057,0.324996,0.557932
4,ABCB11,0.444513,-0.239603,0.220231,-0.227197,-0.089924,0.330269,-0.497336,-0.178940,0.066368,0.367337
...,...,...,...,...,...,...,...,...,...,...,...
4110,ZP1,0.313816,-0.271073,-0.178146,0.078345,0.131971,0.257647,-0.069403,-0.089553,0.168531,-0.556220
4111,ZP4,0.313816,-0.271073,-0.178146,0.078345,0.131971,0.257647,-0.069403,-0.089553,0.168531,-0.556220
4112,ZRANB1,0.203322,-0.054090,0.314513,0.048283,0.207634,-0.036773,-0.000357,0.244518,0.181808,-0.329319
4113,ZWILCH,0.563626,-0.265480,0.322560,0.009124,0.007437,0.024078,-0.185287,0.230829,0.563009,-0.114143


In [107]:
metadata.to_csv("/Users/polina/genetics_gsea/data/for_umap/tensor/metadata.tsv", index=False, sep="\t")
tem.iloc[:, 1:].to_csv("/Users/polina/genetics_gsea/data/for_umap/tensor/coordinates.tsv", index=False, sep="\t", header=False)

In [122]:
output_dir="/Users/polina/genetics_gsea/data/for_umap"

perform_umap_clustering_pandas(metadata, 
                               tem, 
                               output_dir,
                               n_neighbors=15,
                               min_dist=0.9,
                               min_cluster_size=16,
                               umap_dimensions=2)

Merged metadata and coordinates: 4115 entries.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/umap_.py:1865: UserWarning:

using precomputed metric; inverse_transform will be unavailable

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



✅ Updated metadata with clusters saved to: /Users/polina/genetics_gsea/data/for_umap/metadata_clusters_poincare.tsv


'/Users/polina/genetics_gsea/data/for_umap/metadata_clusters_poincare.tsv'

### Plot UMAP

In [90]:
from scipy.spatial.distance import pdist, squareform
import umap.umap_ as umap
import pandas as pd
import hdbscan
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import os

In [95]:
def build_umap_plot_2d_clusters_from_metadata(metadata_cluster_file):
    """
    Builds a 2D UMAP plot from metadata with 'UMAP 1', 'UMAP 2', and 'cluster' columns.
    """
    import pandas as pd
    import plotly.graph_objects as go
    import plotly.express as px

    # Load metadata
    metadata = pd.read_csv(metadata_cluster_file, sep='\t')

    import colorcet as cc  # for colorblind-friendly palettes
    import random

    # Combine multiple colorblind-safe palettes
    safe_colors = (
        cc.glasbey[:25]  # Glasbey is great for many distinguishable categories
    )

    # Shuffle for aesthetics if desired
    random.seed(42)  # ensure reproducibility
    random.shuffle(safe_colors)

    unique_clusters = sorted(metadata['cluster'].unique())
    cluster_color_dict = {
        cluster: ('gainsboro' if cluster == -1 else safe_colors[i % len(safe_colors)])
        for i, cluster in enumerate(unique_clusters)
    }
    metadata['cluster_color'] = metadata['cluster'].map(cluster_color_dict)


    # Build plot
    fig = go.Figure()

    for cluster_id in unique_clusters:
        cluster_df = metadata[metadata['cluster'] == cluster_id]
        # cluster_name = cluster_df['clusterNameReactome'].iloc[0] if cluster_id != -1 else 'Noise'
        color = cluster_color_dict[cluster_id]
        # legend_name = f"{cluster_name}" if cluster_id != -1 else 'Noise'

        # # Marker outline only for drug targets (not visible in legend)
        # marker_lines = [
        #     dict(width=1, color='black') if val != 0 else dict(width=0)
        #     for val in cluster_df['geneticAD']
        # ]

        fig.add_trace(go.Scatter(
            x=cluster_df['UMAP 1'],
            y=cluster_df['UMAP 2'],
            mode='markers',
            # name=legend_name,
            marker=dict(
                size=5,
                color=color,
                line=dict(width=0),  # Disable uniform outline to prevent black circles in legend
                opacity=0.7
            ),
            customdata=cluster_df[['cluster', 'approvedSymbol']],
            hovertemplate=(
                "Cluster %{customdata[0]}: %{customdata[1]}<br>" +
                "approvedSymbol: %{customdata[2]}<extra></extra>"
            ),
            showlegend=True
        ))

        # # Add labels for maxPhase > 2
        # high_val_df = cluster_df[cluster_df['maxPhaseChEMBL'] > 2].copy()
        # y_offset = 0.15
        # fig.add_trace(go.Scatter(
        #     x=high_val_df['UMAP 1'],
        #     y=high_val_df['UMAP 2'] + y_offset,
        #     mode='text',
        #     text=high_val_df['approvedSymbol'],
        #     textposition='top center',
        #     textfont=dict(color='black', size=12),
        #     showlegend=False,
        #     hoverinfo='skip'
        # ))

    # # Add dummy legend entry for drug targets
    # fig.add_trace(go.Scatter(
    #     x=[None],
    #     y=[None],
    #     mode='markers',
    #     marker=dict(
    #         symbol='circle-open',
    #         color='black',
    #         size=5,
    #         line=dict(width=1)
    #     ),
    #     # name='– drug target (Pharmaprojects)',
    #     name='– NeuroFlux hit',
    #     showlegend=True
    # ))

    fig.update_layout(
        xaxis=dict(title='UMAP1', showgrid=False, zeroline=False, constrain='domain', ticks='outside', showline=True, mirror=True),
        yaxis=dict(title='UMAP2', showgrid=False, zeroline=False, scaleanchor='x', scaleratio=1, constrain='domain', ticks='outside', showline=True, mirror=True),
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=40),
        legend=dict(title='Cluster', itemsizing='constant'),
        height=700,
        width=1000,
    )

    return fig

In [124]:
metadata_cluster_file = '/Users/polina/genetics_gsea/data/for_umap/metadata_clusters_poincare.tsv'
fig = build_umap_plot_2d_clusters_from_metadata(metadata_cluster_file)
fig

In [120]:
def build_umap_plot_2d_uniqueDiseases(metadata_cluster_file):
    """
    Builds a 2D UMAP plot from metadata with 'UMAP 1', 'UMAP 2', and 'uniqueDiseases' columns.
    Colors are based on the number of uniqueDiseases.
    """
    import pandas as pd
    import plotly.graph_objects as go
    import plotly.express as px

    # Load metadata
    metadata = pd.read_csv(metadata_cluster_file, sep='\t')

    # Build plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=metadata['UMAP 1'],
        y=metadata['UMAP 2'],
        mode='markers',
        marker=dict(
            size=5,
            color=metadata['uniqueDiseases'],  # color by number of uniqueDiseases
            colorscale='Viridis_r',              # you can try 'Plasma', 'Cividis', 'Turbo', etc.
            colorbar=dict(title='Unique Diseases'),
            opacity=0.7,
        ),
        customdata=metadata[['cluster', 'approvedSymbol', 'uniqueDiseases']],
        hovertemplate=(
            "Cluster %{customdata[0]}<br>" +
            "approvedSymbol: %{customdata[1]}<br>" +
            "uniqueDiseases: %{customdata[2]}<extra></extra>"
        ),
        showlegend=False  # no need for cluster legend anymore
    ))

    fig.update_layout(
        xaxis=dict(title='UMAP1', showgrid=False, zeroline=False, constrain='domain', 
                   ticks='outside', showline=True, mirror=True),
        yaxis=dict(title='UMAP2', showgrid=False, zeroline=False, scaleanchor='x', 
                   scaleratio=1, constrain='domain', ticks='outside', showline=True, mirror=True),
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=40),
        height=700,
        width=1000,
    )

    return fig


In [123]:
metadata_cluster_file = '/Users/polina/genetics_gsea/data/for_umap/metadata_clusters_poincare.tsv'
fig = build_umap_plot_2d_uniqueDiseases(metadata_cluster_file)
fig